In [275]:
from mnist import MNIST
from scipy.stats import norm
import numpy as np
import pandas as pd
import math
from tqdm import tqdm
tqdm.pandas()

## Read train and test data

In [276]:
# http://yann.lecun.com/exdb/mnist/

mndata = MNIST('data/mnist_data_files')
mndata.gz=True
train_images, train_labels = mndata.load_training()

### process data and threashold

In [277]:
train_images = np.array(train_images)
train_labels = np.array(train_labels)

In [278]:
train_images = (pd.DataFrame(train_images) > 127).astype(np.int)
train_labels = pd.DataFrame(train_labels,columns=["label"])

In [279]:
# 60000 rows 28*28 pixels
print(train_images.shape) 
print(train_labels.shape)

(60000, 784)
(60000, 1)


In [280]:
value_counts = train_labels["label"].value_counts(normalize=True)
p_train_labels = pd.DataFrame()
p_train_labels['label'] = value_counts.index
p_train_labels['probability'] = value_counts.values

In [281]:
p_train_labels.head(10)

,label,probability
0,1,0.112367
1,7,0.104417
2,3,0.102183
3,2,0.099300
4,9,0.099150
5,0,0.098717
6,6,0.098633
7,8,0.097517
8,4,0.097367
9,5,0.090350


In [282]:
test_images, test_labels = mndata.load_testing()

In [283]:
test_images = np.array(test_images)
test_labels = np.array(test_labels)

In [284]:
test_images = (pd.DataFrame(test_images) > 127).astype(np.int)
test_labels = pd.DataFrame(test_labels,columns=["label"])

In [285]:
print(test_images.shape)
print(test_labels[1:10])

(10000, 784)
   label
1      2
2      1
3      0
4      4
5      1
6      4
7      9
8      5
9      9


In [286]:
train_df = train_images.join(train_labels)
test_df = test_images.join(test_labels)
test_df.head()

,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,label
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,7
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4


In [287]:
def get_params(label_group):
    images_df = label_group.drop(['label'], axis=1)
    return images_df.apply(lambda x: np.asarray(norm.fit(x)), axis=0)

label_params = train_df.groupby(['label']).apply(get_params)

In [288]:
assert label_params.shape == (20, 784)
label_params.shape

(20, 784)

In [289]:
# label_params.loc[0, :].loc[0]

## Naive Bayes - normal distribution - untouched

In [290]:
# params_ = np.apply_along_axis(lambda x: norm.fit(x), 1, train_images.T)

In [291]:
def calculate_likelihood_for_each_label(p_label, feature_vec, params):
    means = params.loc[0]
    stds = params.loc[1]
    likelihood = np.nansum(norm.logpdf(feature_vec, means, stds))
    likelihood = likelihood + np.log(p_label['probability'])
    return np.array([p_label['label'], likelihood])
    
def get_predict(likelihoods):
    max_row = [float("-inf"), float("-inf")]
    for likelihood in likelihoods:
        if(likelihood[1] > max_row[1]):
            max_row = likelihood
    return max_row[0]

### Evaluate

In [292]:
def predict(image):
    likelihoods = []
    for index, p_train_label in p_train_labels.iterrows():
        params = label_params.loc[p_train_label['label'], :]
        likelihoods.append(calculate_likelihood_for_each_label(p_train_label, image, params))
        
    return get_predict(np.array(likelihoods))

# predict(test_images.loc[1])


In [293]:
predicts = test_images.progress_apply(predict, axis=1)


  0%|          | 0/10000 [00:00<?, ?it/s]/Users/qingemeng/Documents/dev/cs498aml/env/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1704: RuntimeWarning: divide by zero encountered in true_divide
  x = np.asarray((x - loc)/scale, dtype=dtyp)
/Users/qingemeng/Documents/dev/cs498aml/env/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1704: RuntimeWarning: invalid value encountered in true_divide
  x = np.asarray((x - loc)/scale, dtype=dtyp)
/Users/qingemeng/Documents/dev/cs498aml/env/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:874: RuntimeWarning: invalid value encountered in greater_equal
  return (self.a <= x) & (x <= self.b)
/Users/qingemeng/Documents/dev/cs498aml/env/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:874: RuntimeWarning: invalid value encountered in less_equal
  return (self.a <= x) & (x <= self.b)

 12%|█▏        | 1188/10000 [00:14<01:39, 88.79it/s]


 26%|██▌       | 2608/10000 [00:29<01:23, 88.18it/s]


 41%|████      | 4061/10000 [00:45<00:57, 103.40it/s]


 56%|█████▌    | 5618/10000 [01:01<00:43, 100.79it/s]


 72%|███████▏  | 7186/10000 [01:16<00:27, 103.76it/s]


 88%|████████▊ | 8758/10000 [01:31<00:12, 101.46it/s]


100%|██████████| 10000/10000 [01:43<00:00, 96.19it/s]

In [294]:
predicts.head()

0    7.0
1    2.0
2    1.0
3    0.0
4    4.0
dtype: float64

In [295]:
def calculate_accuracy(actual, predicts):
    TP = 0
    num_total = len(actual)
    for i in range(num_total):
        if actual[i] == predicts[i]:
            TP = TP + 1
    return TP/num_total

In [296]:
calculate_accuracy(np.array(test_labels), np.array(predicts))

0.7824